<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'> Business Loyalty Agent in Teradata: Reasoning Over Policy Documents and Structured Business Data
      
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>


<p style="font-size:20px;font-family:Arial"><b>Introduction:</b></p>

<p style="font-size:16px;font-family:Arial">
In many finance and compliance workflows, the hardest questions aren't solved using SQL alone.
Eligibility rules, policies, and exceptions usually live in unstructured documents, while the facts needed to
evaluate those rules live in structured tables.
</p>

<p style="font-size:16px; font-family:Arial">
This notebook demonstrates how easy it is to build a <b>governed business loyalty agent</b> using
<b>Teradata package for LangChain, Teradata Vector Store, and the Teradata MCP</b>. The agent answers
real business questions by combining:
</p>

<ul style="font-size:16px; font-family:Arial; margin-left:20px; line-height:1.8;">
  <li><b>Unstructured policy definitions</b> stored in a Teradata Vector Store</li>
  <li><b>Structured business data</b> stored in Teradata tables</li>
  <li><b>Pre-approved, governed SQL logic</b> exposed through MCP tools</li>
</ul>

<p style="font-size:16px; font-family:Arial">
Rather than generating SQL or relying on assumptions, the agent is limited
to use only trusted data sources and approved SQL code, which make it suitable for regulated
finance and compliance-sensitive workflows.
</p>

<!-- <div style="text-align:center">
  <img src="./images/governed_agent_architecture.png"
       width="1000"
       alt="Governed Finance Agent Architecture"
       style="border:4px solid #404040; border-radius:10px;">
</div> -->

<p style="font-size:20px;font-family:Arial; margin-top:20px"><b>Business Value:</b></p>
<p style="font-size:16px;font-family:Arial">
The business question we want to answer is:
</p>
<p style="font-size:18px;font-family:Arial; font-style:italic; margin-left:20px;">
“Is a business eligible for a loyalty discount, and why?”
</p>

<p style="font-size:16px;font-family:Arial">
Answering this question requires:
</p>

<ul style="font-size:16px;font-family:Arial; margin-left:20px; line-height:1.8;">
  <li>Retrieving official policy and eligibility rules from documents</li>
  <li>Evaluating business facts such as tenure, status, region, and risk flags from structured tables</li>
  <li>Applying exceptions, exclusions, and effective dates defined in policy</li>
  <li>Producing an explainable answer suitable for audit and compliance review</li>
</ul>

<p style="font-size:16px;font-family:Arial"><b>This demo illustrates how Teradata enables:
:</b></p>

<ul style="font-size:16px;font-family:Arial; margin-left:20px; line-height:1.8;">
  <li><b>Vector search</b> over the official policy documents using Teradata Vector Store via Teradata package for Langchain</li>
  <li><b>Governed SQL</b> execution, only approved MCP tools; no free-form SQL</li>
  <li><b>Agentic reasoning</b> that combines policy text, exceptions, exclusions, and dates with business data</li>
  <li><b>Governance by design</b>the agent never generates SQL and must justify every decision</li>
</ul>

<hr style='height:2px;border:none'>
<b style = 'font-size:20px;font-family:Arial'>1. Configure the environment</b>

In [ ]:
%%capture
!pip install -U langchain-teradata teradata-mcp-server langchain-mcp-adapters teradatagenai langchain langchain-aws --quiet


<div class="alert alert-block alert-info">
<p style = 'font-size:16px;font-family:Arial'><b>Please</b><i> restart the kernel after executing the above cell to include/update these libraries into memory for this kernel. The simplest way to restart the Kernel is by typing zero zero: <b> 0 0</b></i> and then clicking <b>Restart</b>.</p>
</div>

In [ ]:
import os
from getpass import getpass
from teradataml import *
from teradatagenai import TeradataAI, TextAnalyticsAI
from teradataml import create_context, set_auth_token
import time

<hr style="height:2px;border:none">
<p style = 'font-size:20px;font-family:Arial'><b>2. Connect to VantageCloud</b></p>
<p style = 'font-size:16px;font-family:Arial'>Connect to VantageCloud using <code>create_context</code> from the teradataml Python library. This environment has been prepared for connecting to a VantageCloud OAF Container. All the details required have been provided.</p>

<p style = 'font-size:18px;font-family:Arial;'><b>2.1 Load the Environment Variables and Connect to Vantage</b></p>
<p style = 'font-size:16px;font-family:Arial;'>Load the environment variables from a .env file and use them to create a connection context to VantageCloud.</p>


In [ ]:
print("Checking if this environment is ready to connect to VantageCloud Lake...")

if os.path.exists("/home/jovyan/JupyterLabRoot/VantageCloud_Lake/.config/.env"):
    print("Your environment parameter file exist.  Please proceed with this use case.")
    # Load all the variables from the .env file into a dictionary
    env_vars = dotenv_values("/home/jovyan/JupyterLabRoot/VantageCloud_Lake/.config/.env")
    # Create the Context
    eng = create_context(host=env_vars.get("host"), username=env_vars.get("username"), password=env_vars.get("my_variable"))
    execute_sql('''SET query_band='DEMO=Langchain_Teradata_MCP_Agent.ipynb;' UPDATE FOR SESSION;''')
    print("Connected to VantageCloud Lake with:", eng)
else:
    print("Your environment has not been prepared for connecting to VantageCloud Lake.")
    print("Please contact the support team.")

<p style = 'font-size:18px;font-family:Arial;'><b>2.2 Set your Authentication Token for this session.</b></p>
<p style = 'font-size:16px;font-family:Arial;'>Load the required values from the included .env file.</p>

In [ ]:
# We've already loaded all the values into our environment variables and into a dictionary, env_vars.

if set_auth_token(base_url=env_vars.get("ues_uri"),
                  pat_token=env_vars.get("access_token"), 
                  pem_file=env_vars.get("pem_file"),
                  valid_from=int(time.time())
                 ):
    print("UES Authentication successful")
else:
    print("UES Authentication failed. Check credentials.")
    sys.exit(1)

<hr style="height:2px; border:none">

<p style="font-size:20px; font-family:Arial"><b>3. Load the data</b></p>

<p style="font-size:16px; font-family:Arial">
To evaluate loyalty discount eligibility, the agent needs business data (tenure, status, type, region, risk).  
In this step, we use a <code>buisness</code> table of businesss with sample rows designed to demonstrate common policy edge cases.
</p>

<p style="font-size:16px; font-family:Arial">
<b>💼 Use Case Summary:</b><br>
We will answer: <i>“Is a specific business eligible for a loyalty discount, and why?”</i><br>
This requires combining policy rules (from the vector store) with business attributes from the table.
</p>

<p style="font-size:16px; font-family:Arial">
<b>What’s in the <code>businesss</code> table?</b>
</p>

<ul style="font-size:16px; font-family:Arial; margin-left:20px; line-height:1.8;">
  <li><b>business_type</b> – Enterprise / SMB / Gov (drives eligibility and exceptions)</li>
  <li><b>region</b> – NA / EMEA (used for regional exceptions)</li>
  <li><b>status</b> – Active / Suspended (eligibility requires Active)</li>
  <li><b>risk_flag</b> – Y/N (hard exclusion when Y)</li>
  <li><b>start_date</b> – used to compute tenure in months</li>
</ul>

<p style="font-size:16px; font-family:Arial">
<b>Why this sample data matters:</b> each row demonstrates a different branch of the policy logic (eligible, excluded by risk, excluded by region, requires manual approval, etc.).
</p>



In [ ]:
businesses = DataFrame(in_schema("DEMO_Financial", "Business"))
businesses


<p style="font-size:20px; font-family:Arial"><b>4. Configure Amazon Bedrock credentials for embeddings</b></p>

<p style="font-size:16px; font-family:Arial">
Teradata Vector Store uses  <b>text embeddings</b> to perform semantic search over policy documents.
In this step, we configure access to <b>Amazon Bedrock</b> so Teradata can generate embeddings  <i>inside the database</i>.
We provide AWS credentials and register them in Teradata as an <b>AUTHORIZATION object</b>
(<code>bedrockauth</code>). This allows Teradata to call Bedrock models using
the <code>teradatagenai</code> <code>TeradataAI</code> interface without exposing credentials to the agent or embedding logic.
</p>


In [ ]:
os.environ["AWS_ACCESS_KEY_ID"] = getpass("AWS_ACCESS_KEY_ID: ")
os.environ["AWS_SECRET_ACCESS_KEY"] = getpass("AWS_SECRET_ACCESS_KEY: ")
# os.environ["AWS_SESSION_TOKEN"] = getpass("AWS_SESSION_TOKEN (leave blank if none): ")
os.environ["AWS_DEFAULT_REGION"] = getpass("AWS_DEFAULT_REGION (e.g., us-west-2): ")

In [ ]:
key = os.environ["AWS_ACCESS_KEY_ID"]
secret = os.environ["AWS_SECRET_ACCESS_KEY"]
# token = os.environ.get("AWS_SESSION_TOKEN", "").strip()
region = os.environ.get("AWS_DEFAULT_REGION")
qry = f"""
REPLACE AUTHORIZATION bedrockauth
USER '{key}'
PASSWORD '{secret}'
"""
execute_sql(qry)

<p style="font-size:20px; font-family:Arial"><b> 5. Create the embeddings client</b></p>
<p style="font-size:16px; font-family:Arial">We use`TeradataAI` class with **Amazon Titan Embed**. for embedding generation. This will convert both policy text and queries into vectors so the vector store can retrieve the most relevant rules.</p>


In [ ]:
#Instantiate the TeradataAI class with the Amazon Bedrock model for embedding generation.
llm_embedding = TeradataAI(api_type="aws",                      
               model_name="amazon.titan-embed-text-v1",
               authorization = "bedrockauth",
               region="us-west-2"
               )

<p style="font-size:20px; font-family:Arial"><b>6. Build the Teradata Vector Store with Teradata Package for Langchain</b></p>
<p style="font-size:18px; font-family:Arial"><b>6.1. Define the content of the page of an unstructured policy documents.</p>
<p style="font-size:16px; font-family:Arial">
Reference:
  <a href="https://docs.langchain.com/oss/python/integrations/vectorstores/teradata">
    Teradata Vector Store – LangChain Documentation
  </a>
</p>


In [ ]:
from langchain_teradata import TeradataVectorStore
from langchain_core.documents import Document

docs = [
    Document(page_content="Loyalty Discount Policy (v2, effective 2025-10-01): A business is eligible for a 10% loyalty discount if they are Active, Enterprise, and have at least 12 months of tenure."),
    Document(page_content="Exclusions: business with risk_flag = 'Y' are not eligible for loyalty discounts, regardless of tenure or business type."),
    Document(page_content="Regional exception (effective 2025-10-01): SMB Businesses in EMEA are not eligible for loyalty discounts."),
    Document(page_content="Government Businesses: Gov accounts are never auto-approved for discounts. They require manual approval and a compliance review ticket."),
    Document(page_content="Tenure definition: Tenure is measured as months between contract start_date and today. If start_date is less than 12 months ago, the Business is not eligible."),
    Document(page_content="Audit requirement: Any eligibility answer must include the policy version used and which exclusion checks were applied.")
]

<p style="font-size:18px; font-family:Arial"><b>6.2. Create the Teradata Vectore Store.</b></p>
<p style="font-size:16px; font-family:Arial">Teradata Vector Store names must be unique across the database. Here we will append our database user name as a prefix to ensure uniqueness and also add the code to check if it already exists.</p>

In [ ]:
try:
    vs_name = env_vars.get("username")+"_"+"finance_metric_policies"
    vs = TeradataVectorStore(name=vs_name,)
except Exception as e:
    print("Details: ", e)

while True:
    df = vs.status()
    if df is None:
        print("The Teradata VectorStore doesn't exist. Creating it now.")
     
        # Create the vector store
        vs = TeradataVectorStore.from_documents(
            name=vs_name,
            documents=docs,
            embedding=llm_embedding
            )
        print("Vector store is being created!")
        time.sleep(10) #We need a few seconds before the next status check.
    else:
        df = vs.status()
        print(f"Current status: {df}. Waiting 10 seconds...")
        time.sleep(10)
        if df is not None:
            break

print(f"The Vector Store Database already exist or has been successfully created!")    


In [ ]:
vs.get_details()

<p style="font-size:20px; font-family:Arial">
  <b>7. Create a Retriever (RAG)</b>
</p>

<p style="font-size:16px; font-family:Arial">
  The <code>vs.as_retriever()</code> method converts the Teradata Vector Store into a
  simple retrieval interface that can be used in agent workflows.
</p>

In [ ]:
# Create a retriever for our RAG Agent pipeline
retriever = vs.as_retriever(search_type="similarity")

# Test the retriever
retrieved_docs = retriever.invoke("Tell me about discounts")

print("Retrieved documents for RAG:")
for doc in retrieved_docs:
    print(f"- {doc.page_content}")

<p style="font-size:20px; font-family:Arial"><b>8.  Wrap Retrieval as a tool the agent can call</b></p>
<p style="font-size:16px; font-family:Arial">Agents select from *tools*.
So we wrap the retriever as a small LangChain tool called `retrieve_kb`.
This controls the agent so that it must retrieve the policy text first and it can’t “guess” the rules.</p>



In [ ]:
from langchain_core.tools import tool

def format_docs(docs):
    return "\n\n".join(d.page_content for d in docs)

@tool
def retrieve_kb(query: str) -> str:
    """Retrieve official metric definitions and governance rules."""
    docs = retriever.invoke(query)
    return format_docs(docs)


<p style="font-size:20px; font-family:Arial"><b>9. Configure the governed SQL tools (Teradata MCP)</b></p>

<p style="font-size:16px; font-family:Arial">
MCP lets us expose <b>pre-approved SQL</b> as named tools.<br>
This is where the agent can call these tools and execute the pre-approved SQL. 
</p>

<p style="font-size:16px; font-family:Arial">
Run the next 3 cells to create a local <code>mcp_config/</code> folder and write two files for this demo:
</p>

<ul style="font-size:16px; font-family:Arial; margin-left:20px;">
  <li><code>profiles.yml</code>: which tools the agent is allowed to use</li>
  <li><code>finance_objects.yml</code>: the governed SQL tools (approved queries)</li>
</ul>


In [ ]:
CONFIG_DIR = "/home/jovyan/JupyterLabRoot/VantageCloud_Lake/UseCases/Governed_Langchain_Teradata_Agent/mcp_config"

os.makedirs(CONFIG_DIR, exist_ok=True)

print("mcp_config directory ready")

<p style="font-size:20px; font-family:Arial"><b>10. Define the “approved SQL” tool</b></p>

<p style="font-size:16px; font-family:Arial">
  We create a <code>finance_objects.yml</code> file to define the custom tools exposed to the agent.
  In this demo, we define <code>finance_check_business_loyalty_eligibility</code>, a tool with
  pre-defined SQL that returns only the business facts required for the policy decision:
</p>

<ul style="font-size:16px; font-family:Arial; margin-left:20px; line-height:1.6;">
  <li>status, type, region, risk flag</li>
  <li>tenure (computed from <code>start_date</code> and <code>CURRENT_DATE</code>)</li>
  <li><code>CURRENT_DATE</code> is retrieved from the database and is not stored as a table column</li>
</ul>

<p style="font-size:16px; font-family:Arial">
  The agent uses this output to evaluate if a business is eligible and why or why not without generating SQL.
</p>

In [ ]:
%%writefile /home/jovyan/JupyterLabRoot/VantageCloud_Lake/UseCases/Governed_Langchain_Teradata_Agent/mcp_config/finance_objects.yml
finance_check_business_loyalty_eligibility:
  type: tool
  description: Return business attributes needed to evaluate loyalty discount eligibility (status, type, region, risk, tenure).
  parameters:
    business_name:
      type: string
      description: Exact business_name as stored in DEMO_Financial.Business.
      required: true
  sql: |
    SELECT
      business_id,
      business_name,
      business_type,
      region,
      status,
      risk_flag,
      ((EXTRACT(YEAR FROM CURRENT_DATE) - EXTRACT(YEAR FROM start_date)) * 12
        + (EXTRACT(MONTH FROM CURRENT_DATE) - EXTRACT(MONTH FROM start_date))) AS tenure_months
    FROM "DEMO_Financial"."Business"
    WHERE business_name = :business_name;


<p style="font-size:20px; font-family:Arial"><b>11. Define Profiles</b></p>

<p style="font-size:16px; font-family:Arial">
  We create a <code>profiles.yml</code> file with a <code>finance_analyst</code> profile to control which MCP tools are available to the agent.
  This profile explicitly exposes the custom tool we built and restricts everything else.
</p>

<p style="font-size:16px; font-family:Arial">
  In this demo, the profile limits the agent to:
</p>

<ul style="font-size:16px; font-family:Arial; margin-left:20px; line-height:1.6;">
  <li>The approved SQL tool <code>finance_check_business_loyalty_eligibility</code> for business eligibility checks</li>
  <li>Optional base discovery tools (database and table listing)</li>
</ul>

In [ ]:
%%writefile /home/jovyan/JupyterLabRoot/VantageCloud_Lake/UseCases/Governed_Langchain_Teradata_Agent/mcp_config/profiles.yml
finance_analyst:
  tool:
    - base_databaseList
    - base_tableList
    - base_readQuery
    - finance_check_business_loyalty_eligibility
  prompt: []
  resource: []



<p style="font-size:20px; font-family:Arial"><b>12. Start MCP and load tools into the agent</b></p>

<p style="font-size:16px; font-family:Arial">
  We use LangChain’s <code>MultiServerMCPClient</code> to start the Teradata MCP  via <code>stdio</code> transport and load only the tools we want to expose to the agent, including custom tools.
  By restricting the tool set instead of loading the full MCP catalog, we keep the agent’s context focused on the task, reduce unnecessary token usage, and improve reliability.
  Each MCP tool is automatically converted into a callable LangChain tool, such as <code>finance_check_business_loyalty_eligibility</code>, which the agent can select at runtime.
</p>


In [ ]:
import asyncio, sys, os
from langchain_mcp_adapters.client import MultiServerMCPClient

TD_HOST = env_vars["host"]
TD_USER = env_vars["username"]
TD_PASSWORD = env_vars["my_variable"]  
TD_DB = TD_USER

DATABASE_URI = f"teradata://{TD_USER}:{TD_PASSWORD}@{TD_HOST}:1025/{TD_DB}"

mcp_env = {
    "DATABASE_URI": DATABASE_URI,
    "MCP_TRANSPORT": "stdio",
}

client = MultiServerMCPClient(
    {
        "teradata": {
            "transport": "stdio",
            "command": sys.executable,
            "args": [
                "-m", "teradata_mcp_server",
                "--profile", "finance_analyst",
                "--config_dir", CONFIG_DIR
            ],
            "env": mcp_env,
            "cwd": CONFIG_DIR,
        }
    }
)

tools = await client.get_tools()
print([t.name for t in tools])




<p style="font-size:20px; font-family:Arial"><b>13. Initialize the LLM </b></p>


In [ ]:
from langchain.chat_models import init_chat_model

llm = init_chat_model(
    model="anthropic.claude-3-haiku-20240307-v1:0",
    model_provider="bedrock_converse",
    aws_access_key_id=key,
    aws_secret_access_key=secret
)

<p style="font-size:20px; font-family:Arial"><b>14. Create the Agent </b></p>


In [ ]:
from langchain.agents import create_agent

agent = create_agent(
    model=llm,
    tools=[retrieve_kb] + tools,
    system_prompt = (
    "You are a compliance-aware finance assistant.\n"
    "RULES (must follow):\n"
    "1) You MUST call retrieve_kb before you make any eligibility decision.\n"
    "2) You MUST call finance_check_business_loyalty_eligibility to fetch business facts.\n"
    "3) You may ONLY use policy clauses that appear verbatim (or clearly paraphrased) in retrieve_kb output.\n"
    "4) If retrieve_kb was not called, say you cannot answer.\n"
    "5) If the SQL tool returns 0 rows or an error, say you cannot answer and ask for the exact business_name.\n"
    "Never generate SQL.\n"
),
)


<p style="font-size:20px; font-family:Arial"><b>15. Test the QnA Agent </b></p>
<p style="font-size:16px; font-family:Arial">Here are some sample questions to ask:</p>
<ul style="font-size:16px; font-family:Arial">
    <li>How much is the loyalty discount for businesses?</li>
</ul>

In [ ]:
from langchain_core.messages import HumanMessage
from pprint import pprint

user_question = input("Enter your question for the governed agent:\n> ").strip()

result = await agent.ainvoke(
    {"messages": [HumanMessage(content=user_question)]}
)

for m in result["messages"]:
    t = getattr(m, "type", "")
    if t == "human":
        print("\nQUESTION:\n", m.content)
    elif t == "tool" and m.name == "retrieve_kb":
        print("\nVECTOR STORE DEFINITION USED:\n", m.content.split("\n\n")[0])
    elif t == "tool" and m.name == "finance_check_business_loyalty_eligibility":
        print("\nDB BUSINESS RESULT (raw tool output):")
        pprint(m.content)
    elif t == "ai" and isinstance(m.content, str) and m.content.strip():
        print("\nGOVERNED AGENT ANSWER:\n", m.content)

<p style="font-size:20px; font-family:Arial"><b>16.Run a direct query to confirm Agent Answers</b></p>

<p style="font-size:16px; font-family:Arial">
You can  validate the agent’s answer by running a direct SQL query using the same business name provided in the user question. This query returns the authoritative business facts stored in Teradata. These values should match the facts the agent used when applying the loyalty discount policy rules:
</p>

<ul style="font-size:16px; font-family:Arial; margin-top:0;">
  <li><code>business_type</code></li>
  <li><code>status</code></li>
  <li><code>region</code></li>
  <li><code>risk_flag</code></li>
  <li><code>tenure_months</code></li>
</ul>


In [ ]:
result = execute_sql(f"""
    SELECT
        business_id,
        business_name,
        business_type,
        region,
        status,
        risk_flag,
        /* tenure in months */
        ((EXTRACT(YEAR FROM CURRENT_DATE) - EXTRACT(YEAR FROM start_date)) * 12
          + (EXTRACT(MONTH FROM CURRENT_DATE) - EXTRACT(MONTH FROM start_date))) AS tenure_months
    FROM DEMO_Financial.Business
    WHERE business_name = 'Acme Co';
""")

for row in result:
    print(row)




<hr style="height:2px;border:none">
<b style = 'font-size:20px;font-family:Arial'>15. Cleanup</b>
<p style = 'font-size:16px;font-family:Arial'>Call the destroy() method of the VS object to clean up the objects created during this demo. Creating a dataframe from the result of the <code>vs.status()</code> call is a simple way to check for the existance of the Vector Store.</p>

In [ ]:
while True:
    df = vs.status()
    if df is None:
        break
    else:
        vs.destroy()
        print(f"Current status: {df}. Waiting 10 seconds...")
        time.sleep(10)
        df = vs.status()

print(f"The Vector Store Database has been successfully destroyed!")

In [ ]:
remove_context()